In [16]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction to CS203

## To sort or not to sort?

Consider the following code snippet, what the performance looks like when option is set to 1 or 0?

In [5]:
render_code("calculate_sum.c")

// calculate_sum.c:1-15 (15 lines)
#ifdef __cplusplus
extern "C" {
#endif
long long calculate_sum(int *data, unsigned int size, int threshold)
{
    long long sum=0;
    for (unsigned i = 0; i < size; ++i) {
        if (data[i] >= threshold)
                sum ++;
    }
    return sum;
}
#ifdef __cplusplus
}
#endif

In [6]:
render_code("arraySort.cpp")

// arraySort.cpp:1-37 (37 lines)
#include <algorithm>
#include <ctime>
#include <iostream>
#include <climits>

#ifdef __cplusplus
extern "C" {
#endif
long long calculate_sum(int *data, unsigned int size, int threshold);
#ifdef __cplusplus
}
#endif


int main(int argc, char **argv)
{
    // generate data
//    const unsigned arraySize = 262144;
    int arraySize = atoi(argv[1]);
    int data[arraySize];
    long long sum = 0;

    int iterations = atoi(argv[2]);
    int option = atoi(argv[3]);

    for (unsigned c = 0; c < arraySize; ++c)
        data[c] = std::rand();

    if(option)
        std::sort(data, data + arraySize);

    for (unsigned i = 0; i < iterations; ++i) {
        sum += calculate_sum(data, arraySize, std::rand());
    }
    std::cout << "sum = " << sum << std::endl;
}

In [7]:
! make clean; make

rm -f madd arraySort bitonic quicksort mm
gcc madd.c -o madd 
gcc calculate_sum.c -c 
g++ -O0 arraySort.cpp calculate_sum.o -o arraySort
/usr/local/cuda/bin/nvcc -O3 bitonic.cu -o bitonic
/usr/local/cuda/bin/nvcc -O3 quicksort.cu -o quicksort
gcc -O3 mm.c perfstats.o -o mm 


In [10]:
! time ./arraySort 262144 1000 1

/bin/bash: line 1: 1403241 Segmentation fault      (core dumped) ./arraySort 2621440 1000 1

real	0m0.087s
user	0m0.000s
sys	0m0.006s


In [11]:
! time ./arraySort 262144 1000 0

sum = 127161197

real	0m0.572s
user	0m0.572s
sys	0m0.000s


## Matrix Multiplications

Matrix multiplications in general has an O($n^3$) complexity, how does the complexity work on real computers?

In [17]:
render_code("mm.c", show=["//START","//END"])

// mm.c:71-87 (17 lines)
//START
void mm(double **a, double **b, double **c)
{
  int i,j,k;
  for(i = 0; i < ARRAY_SIZE; i++)
  {
    for(j = 0; j < ARRAY_SIZE; j++)
    {
      for(k = 0; k < ARRAY_SIZE; k++)
      {
        c[i][j] += a[i][k]*b[k][j];
      }
    }
  }
  return;
}
//END

In [13]:
! make mm
! time ./mm 1024

make: 'mm' is up to date.
2.117810 seconds

real	0m2.129s
user	0m2.124s
sys	0m0.004s


In [15]:
! time ./mm 2048

24.032001 seconds

real	0m24.054s
user	0m24.029s
sys	0m0.024s


## Algorithm on GPUs?
Consider quicksort and bitsort, which performs better on GPUs?

### Quick sort

In [25]:
render_code("quicksort.cu", show="quickSortIterative")

// quicksort.cu:62-99 (38 lines)
void quickSortIterative (float arr[], int l, int h)
{
    int lstack[ h - l + 1 ], hstack[ h - l + 1];
    float *d_d;
    int top = -1, *d_l, *d_h;
 
    lstack[ ++top ] = l;
    hstack[ top ] = h;

    cudaMalloc(&d_d, (h-l+1)*sizeof(float));
    cudaMemcpy(d_d, arr,(h-l+1)*sizeof(float),cudaMemcpyHostToDevice);

    cudaMalloc(&d_l, (h-l+1)*sizeof(int));
    cudaMemcpy(d_l, lstack,(h-l+1)*sizeof(int),cudaMemcpyHostToDevice);

    cudaMalloc(&d_h, (h-l+1)*sizeof(int));
    cudaMemcpy(d_h, hstack,(h-l+1)*sizeof(int),cudaMemcpyHostToDevice);
    int n_t = 1;
    int n_b = 1;
    int n_i = 1; 
    while ( n_i > 0 )
    {
        partition<<<n_b,n_t>>>( d_d, d_l, d_h, n_i);
        int answer;
        cudaMemcpyFromSymbol(&answer, d_size, sizeof(int), 0, cudaMemcpyDeviceToHost); 
        if (answer < 1024)
          {
            n_t = answer;
          }
        else
          {
            n_t = 1024;
            n_b = answer/n_t + (answer%n_t==0?0:1);
          }
        n_i = answer;
        cudaMemcpy(arr, d_d,(h-l+1)*sizeof(int),cudaMemcpyDeviceToHost);
    }
}

In [26]:
render_code("quicksort.cu", show="partition")

// quicksort.cu:21-60 (40 lines)
__global__ void partition (float *arr, int *arr_l, int *arr_h, int n)
{
    int z = blockIdx.x*blockDim.x+threadIdx.x;
    d_size = 0;
    __syncthreads();
    if (z<n)
      {
        int h = arr_h[z];
        int l = arr_l[z];
        int x = arr[h];
        int i = (l - 1);
        int temp;
        for (int j = l; j <= h- 1; j++)
          {
            if (arr[j] <= x)
              {
                i++;
                temp = arr[i];
                arr[i] = arr[j];
                arr[j] = temp;
              }
          }
        temp = arr[i+1];
        arr[i+1] = arr[h];
        arr[h] = temp;
        int p = (i + 1);
        if (p-1 > l)
          {
            int ind = atomicAdd(&d_size, 1);
            arr_l[ind] = l;
            arr_h[ind] = p-1;  
          }
        if ( p+1 < h )
          {
            int ind = atomicAdd(&d_size, 1);
            arr_l[ind] = p+1;
            arr_h[ind] = h; 
          }
      }
}

### Bitonic sort

In [27]:
render_code("bitonic.cu", show="bitonic_sort")

// bitonic.cu:80-101 (22 lines)
void bitonic_sort(float *values)
{
  float *dev_values;
  size_t size = NUM_VALS * sizeof(float);

  cudaMalloc((void**) &dev_values, size);
  cudaMemcpy(dev_values, values, size, cudaMemcpyHostToDevice);

  dim3 blocks(BLOCKS,1);    /* Number of blocks   */
  dim3 threads(THREADS,1);  /* Number of threads  */

  int j, k;
  /* Major step */
  for (k = 2; k <= NUM_VALS; k <<= 1) {
    /* Minor step */
    for (j=k>>1; j>0; j=j>>1) {
      bitonic_sort_step<<<blocks, threads>>>(dev_values, j, k);
    }
  }
  cudaMemcpy(values, dev_values, size, cudaMemcpyDeviceToHost);
  cudaFree(dev_values);
}

In [28]:
render_code("bitonic.cu", show="bitonic_sort_step")

// bitonic.cu:48-75 (28 lines)
__global__ void bitonic_sort_step(float *dev_values, int j, int k)
{
  unsigned int i, ixj; /* Sorting partners: i and ixj */
  i = threadIdx.x + blockDim.x * blockIdx.x;
  ixj = i^j;

  /* The threads with the lowest ids sort the array. */
  if ((ixj)>i) {
    if ((i&k)==0) {
      /* Sort ascending */
      if (dev_values[i]>dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
    if ((i&k)!=0) {
      /* Sort descending */
      if (dev_values[i]<dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
  }
}

In [29]:
! ./bitonic

Elapsed time: 0.447s


In [30]:
!./quicksort

Elapsed time: 21.954s


In [31]:
! nvidia-smi -a


==============NVSMI LOG==============

Timestamp                                 : Thu Sep 28 16:19:46 2023
Driver Version                            : 535.54.03
CUDA Version                              : 12.2

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Product Name                          : NVIDIA GeForce RTX 4070
    Product Brand                         : GeForce
    Product Architecture                  : Ada Lovelace
    Display Mode                          : Disabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial 

## What a "real" program look like?

In [18]:
! objdump -D mm


mm:     file format elf64-x86-64


Disassembly of section .interp:

0000000000000318 <.interp>:
 318:	2f                   	(bad)  
 319:	6c                   	insb   (%dx),%es:(%rdi)
 31a:	69 62 36 34 2f 6c 64 	imul   $0x646c2f34,0x36(%rdx),%esp
 321:	2d 6c 69 6e 75       	sub    $0x756e696c,%eax
 326:	78 2d                	js     355 <_init-0xcab>
 328:	78 38                	js     362 <_init-0xc9e>
 32a:	36 2d 36 34 2e 73    	ss sub $0x732e3436,%eax
 330:	6f                   	outsl  %ds:(%rsi),(%dx)
 331:	2e 32 00             	xor    %cs:(%rax),%al

Disassembly of section .note.gnu.property:

0000000000000338 <.note.gnu.property>:
 338:	04 00                	add    $0x0,%al
 33a:	00 00                	add    %al,(%rax)
 33c:	10 00                	adc    %al,(%rax)
 33e:	00 00                	add    %al,(%rax)
 340:	05 00 00 00 47       	add    $0x47000000,%eax
 345:	4e 55                	rex.WRX push %rbp
 347:	00 02                	add    %al,(%rdx)
 349:	00 00                	ad